In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', 107)
pd.set_option('display.max_rows', 13693)


In [ ]:
# Load the decrypted file into a pandas DataFrame
df = pd.read_excel("/content/USM FY 2014 New FT and Transfers Retention Degree and FA  12-21-23 (1) (1)(1).xlsx")

# Display the first few rows
print(df.head())

    campusid   opeid  FirstTimeStatus  Race_New  AttendStatus  Gender AppZip  \
0  @00147352  210400                5         1             2       2  20705   
1  @00159764  210400                5         3             1       2  20876   
2  @00170784  210400                5         9             2       2  21211   
3  @00186283  210400                5         5             2       2  20852   
4  @00191705  210400                5         5             1       2  20794   

   EntryMajor  EntryDegree  FY2014Fall  FY2015Fall  FY2016Fall  FY2017Fall  \
0      120300           40      2014.0      2015.0         NaN         NaN   
1      121300           40      2014.0      2015.0         NaN         NaN   
2      120300           40      2014.0      2015.0         NaN         NaN   
3      120300           40      2014.0      2015.0      2016.0      2017.0   
4      120300           40      2014.0      2015.0         NaN         NaN   

   FY2018Fall  FY2019Fall  FY2020Fall  FY2021Fall 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13694 entries, 0 to 13693
Columns: 107 entries, StudentKey to Y6ProgramAmount
dtypes: float64(31), int64(23), object(53)
memory usage: 11.2+ MB


In [ ]:
df['Plan1_1'].unique()

array(['Mathematics', 'Bioc & Mol Biol', 'Mech Eng', 'Biol Sci',
       'Info Sys', 'Comp Eng', 'Environ Sci', 'Chemical Eng',
       'Engineering', 'Comp Sci', 'Chemistry', 'Environ Std', 'Physics',
       'Statistics', 'Binf & Comp Bio', 'Bus Tech Admin',
       'Chemistry Educ', 'Physics Educ', 'Pre-Engineering',
       'Pre-Chem Eng', 'Pre-Comp Sci', 'Pre-Comp Eng', 'Pre-Mech Eng',
       'Biol Educ', 'Geog & Env Std', 'Environ Sci & Geog',
       'Trnsltnl Life Sci Tech'], dtype=object)

## Total count of students in a dataset:**13694**
## Total number of students Graduated: **9388**
## Total number of students not Graduated: **4306**

## Total number students whose initial major and final reported major are the same(regardless of graduation status):**6750**

## Total number of students whose initial major and final reported major are not the same (regardless of graduation status): **6944**
## Total number students whose initial major and final reported major are the same and grduated :**4049**

## Total number of students whose initial major and final reported major are not the same and grduated :**5339**


In [ ]:
# Count students who graduated
graduated_count = df[df['Graduated'] == 'Yes']['StudentKey'].nunique()

# Count students who did not graduate
not_graduated_count = df[df['Graduated'] != 'Yes']['StudentKey'].nunique()

# Display the results
graduation_summary = {
    "Total Graduated Students": graduated_count,
    "Total Not Graduated Students": not_graduated_count
}

graduation_summary

{'Total Graduated Students': 9388, 'Total Not Graduated Students': 4306}

In [ ]:
# Create a new column to check if the initial major (Plan1_1) is the same as the final reported major (LastrptgPlanShortDescr)
df['Same_Initial_Final_Major'] = df.apply(lambda x: 1 if str(x['Plan1_1']).strip().lower() == str(x['LastrptgPlanShortDescr']).strip().lower() else 0, axis=1)

# Count how many students kept their original major versus those who changed
same_major_count = df['Same_Initial_Final_Major'].sum()
changed_major_count = len(df) - same_major_count

# Display the results
major_change_summary = {
    "Students Who Kept Their Initial Major": same_major_count,
    "Students Who Changed Their Major": changed_major_count
}

major_change_summary


{'Students Who Kept Their Initial Major': np.int64(6750),
 'Students Who Changed Their Major': np.int64(6944)}

In [ ]:
# Filter only students who graduated
graduated_df = df[df['Graduated'] == 'Yes']

# Create a new column to check if the initial major (Plan1_1) is the same as the final reported major (LastrptgPlanShortDescr) for graduates
graduated_df['Same_Initial_Final_Major'] = graduated_df.apply(
    lambda x: 1 if str(x['Plan1_1']).strip().lower() == str(x['LastrptgPlanShortDescr']).strip().lower() else 0, axis=1
)

# Count how many graduates kept their original major versus those who changed
same_major_count = graduated_df['Same_Initial_Final_Major'].sum()
changed_major_count = len(graduated_df) - same_major_count

# Display the results
graduated_major_change_summary = {
    "Graduates Who Kept Their Initial Major": same_major_count,
    "Graduates Who Changed Their Major": changed_major_count
}

graduated_major_change_summary

<ipython-input-7-dfa370a04de3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  graduated_df['Same_Initial_Final_Major'] = graduated_df.apply(


{'Graduates Who Kept Their Initial Major': np.int64(4049),
 'Graduates Who Changed Their Major': np.int64(5339)}

## **Individual count of Students who started and final reported major are same or not (regardless of graduation status)**

In [ ]:
# Count of students who started and ended with the same major
same_major_counts = df[df['Same_Initial_Final_Major'] == 1].groupby('Plan1_1')['StudentKey'].count()

# Count of students who started with one major but ended with another
changed_major_counts = df[df['Same_Initial_Final_Major'] == 0].groupby(['Plan1_1', 'LastrptgPlanShortDescr'])['StudentKey'].count()

# Convert results to DataFrames for better readability
same_major_df = same_major_counts.reset_index().rename(columns={'StudentKey': 'Count_Started_Ended_Same_Major'})
changed_major_df = changed_major_counts.reset_index().rename(columns={'StudentKey': 'Count_Changed_Major'})

same_major_df



,Plan1_1,Count_Started_Ended_Same_Major
0,Binf & Comp Bio,41
1,Bioc & Mol Biol,465
2,Biol Educ,8
3,Biol Sci,2390
4,Bus Tech Admin,337
5,Chemical Eng,188
6,Chemistry,120
7,Chemistry Educ,5
8,Comp Eng,196
9,Comp Sci,614


In [ ]:
changed_major_df

,Plan1_1,LastrptgPlanShortDescr,Count_Changed_Major
0,Binf & Comp Bio,Afric Std,1
1,Binf & Comp Bio,Bioc & Mol Biol,6
2,Binf & Comp Bio,Biol Sci,21
3,Binf & Comp Bio,Bus Tech Admin,4
4,Binf & Comp Bio,Chemical Eng,1
5,Binf & Comp Bio,Chemistry,1
6,Binf & Comp Bio,Comp Eng,1
7,Binf & Comp Bio,Comp Sci,7
8,Binf & Comp Bio,Economics,1
9,Binf & Comp Bio,Environ Sci,1


## **Individual count of Students who started and final reported major and graduated are same or not**

In [ ]:
# Count of students who started and ended with the same major among graduates
same_major_counts = graduated_df[graduated_df['Plan1_1'] == graduated_df['LastrptgPlanShortDescr']].groupby('Plan1_1')['StudentKey'].count()

# Count of students who started with one major but ended with another among graduates
changed_major_counts = graduated_df[graduated_df['Plan1_1'] != graduated_df['LastrptgPlanShortDescr']].groupby(['Plan1_1', 'LastrptgPlanShortDescr'])['StudentKey'].count()

# Convert results to DataFrames for better readability
same_major_df = same_major_counts.reset_index().rename(columns={'StudentKey': 'Count_Started_Ended_Same_Major'})
changed_major_df = changed_major_counts.reset_index().rename(columns={'StudentKey': 'Count_Changed_Major'})

same_major_df



,Plan1_1,Count_Started_Ended_Same_Major
0,Binf & Comp Bio,27
1,Bioc & Mol Biol,297
2,Biol Educ,3
3,Biol Sci,1624
4,Bus Tech Admin,174
5,Chemical Eng,131
6,Chemistry,66
7,Chemistry Educ,3
8,Comp Eng,115
9,Comp Sci,379


In [ ]:
changed_major_df

,Plan1_1,LastrptgPlanShortDescr,Count_Changed_Major
0,Binf & Comp Bio,Afric Std,1
1,Binf & Comp Bio,Bioc & Mol Biol,5
2,Binf & Comp Bio,Biol Sci,17
3,Binf & Comp Bio,Bus Tech Admin,3
4,Binf & Comp Bio,Chemical Eng,1
5,Binf & Comp Bio,Chemistry,1
6,Binf & Comp Bio,Comp Eng,1
7,Binf & Comp Bio,Comp Sci,6
8,Binf & Comp Bio,Economics,1
9,Binf & Comp Bio,Environ Sci,1


# **STEM MAJORS**

In [ ]:


# Define the list of STEM majors based on the provided document
stem_majors = [
    "Mathematics", "Bioc & Mol Biol", "Mech Eng", "Biol Sci",
    "Info Sys", "Comp Eng", "Environ Sci", "Chemical Eng",
    "Engineering", "Comp Sci", "Chemistry", "Environ Std",
    "Physics", "Statistics", "Binf & Comp Bio", "Bus Tech Admin",
    "Chemistry Educ", "Physics Educ", "Pre-Engineering",
    "Pre-Chem Eng", "Pre-Comp Sci", "Pre-Comp Eng", "Pre-Mech Eng",
    "Biol Educ", "Geog & Env Std", "Environ Sci & Geog",
    "Trnsltnl Life Sci Tech"
]

# Count total students
total_students = df['StudentKey'].nunique()

# Count students who started in STEM (Plan1_1 in STEM majors)
students_started_stem = df[df['Plan1_1'].isin(stem_majors)]['StudentKey'].nunique()

# Count students who started in STEM and graduated
students_started_stem_graduated = df[
    (df['Plan1_1'].isin(stem_majors)) & (df['Graduated'] == 'Yes')
]['StudentKey'].nunique()

# Count students who started in STEM, ended in STEM, and graduated
students_started_ended_stem_graduated = df[
    (df['Plan1_1'].isin(stem_majors)) &  # Started in STEM
    (df['LastrptgPlanShortDescr'].isin(stem_majors)) &  # Ended in STEM
    (df['Graduated'] == 'Yes')  # Graduated
]['StudentKey'].nunique()

# Count students who started in STEM, ended in NOT STEM, and graduated
students_started_stem_ended_nonstem_graduated = df[
    (df['Plan1_1'].isin(stem_majors)) &  # Started in STEM
    (~df['LastrptgPlanShortDescr'].isin(stem_majors)) &  # Ended in NON-STEM
    (df['Graduated'] == 'Yes')  # Graduated
]['StudentKey'].nunique()

# Display the results
summary_counts = {
    "Total Students": total_students,
    "Students Who Started in STEM": students_started_stem,
    "Students Who Started in STEM and Graduated(Any Major)": students_started_stem_graduated,
    "Students Who Started, Ended in STEM, and Graduated": students_started_ended_stem_graduated,
    "Students Who Started in STEM, Ended in Non-STEM, and Graduated": students_started_stem_ended_nonstem_graduated
}

summary_counts


{'Total Students': 13694,
 'Students Who Started in STEM': 13694,
 'Students Who Started in STEM and Graduated(Any Major)': 9388,
 'Students Who Started, Ended in STEM, and Graduated': 7929,
 'Students Who Started in STEM, Ended in Non-STEM, and Graduated': 1459}

# **Support**

In [ ]:
df['totalsupport'] = df.loc[:,'Y1GrantAmount':'Y6MeritAmount'].sum(1)
df['supportedY1'] = df.apply(lambda x: 1 if (x['Y1GrantAmount'] + x['Y1MeritAmount'] != 0) else 0, axis = 1)
df['supported'] = df.apply(lambda x: 1 if (x['totalsupport'] != 0) else 0, axis = 1)
df[['supported']].value_counts()

,count
supported,
1,8452
0,5242


## **Financial need category (Y1Need) and support status (supported) for STEM students**

In [ ]:
# Filter students who started in STEM, ended in STEM, and graduated
stem_students = df[
    (df['Plan1_1'].isin(stem_majors)) &  # Started in STEM
    (df['LastrptgPlanShortDescr'].isin(stem_majors)) &  # Ended in STEM
    (df['Graduated'] == 'Yes')  # Graduated
]

# Group by financial need category (Y1Need) and support status (supported) for these students
support_by_need_stem = stem_students.groupby(['Y1Need', 'supported'])['StudentKey'].count().reset_index()

support_by_need_stem = support_by_need_stem.rename(columns={
    'Y1Need': 'Financial Need Category',
    'supported': 'Support Status',
    'StudentKey': 'Student Count'
})

support_by_need_stem


,Financial Need Category,Support Status,Student Count
0,High,0,55
1,High,1,2429
2,Low,0,994
3,Low,1,1427
4,Medium,0,328
5,Medium,1,692
6,No Fafsa,0,1279
7,No Fafsa,1,725


In [ ]:
# Function to determine if any degree date columns are not null (indicating graduation)
def check_graduation(row):
    # Check each degree date column for non-null values
    for i in range(1, 5):  # Assuming you have degreedate1 to degreedate4
        if pd.notnull(row[f'degreedate{i}']):
            return 'Yes'
    return 'No'

#Function to see if the degree obtained was in STEM
hegisstem = ('7','9','4','17','19');

myctr = 0;
def check_stem(row):
  #ctr = 0;
  if (str(row['EntryMajor']).startswith(hegisstem)):
    #print("found 17");
    return 'Yes';
  #print();
  return 'No';


# Apply the function to create the "Graduated" column
print('Total Students: ', len(df.index));
df['Graduated'] = df.apply(check_graduation, axis=1);
print('Total Graduated: ', len(df[df['Graduated'] == 'Yes']))
df['Stem1'] = df.apply(check_stem, axis=1);
#df['stem2stem'] = df.apply(lambda x: 1 if (x['Stem1'] == 'Yes') and (x['HegisDeg1'] == 'Yes') else 0, axis=1)
print('Stem Majors: ', (df.Stem1.values == 'Yes').sum());
print ('Stem Majors Graduated: ', len(df[(df['Stem1'] == 'Yes') & (df['Graduated'] == 'Yes')]))
#print (len(df[(df['Graduated'] == 'Yes']) && (df['Stem1'] == 'Yes')))

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_file.csv', index=False);
#print(df.head(5));




Total Students:  30606
Total Graduated:  17645
Stem Majors:  14151
Stem Majors Graduated:  7774


In [ ]:
# Identify rows where entrydegreeseeking is 'Non-Degree undergrad'
rows_to_drop = df[df['entrydegreeseeking'] == 'Non-Degree undergrad'].index

# Drop these rows from the DataFrame
df.drop(rows_to_drop, inplace=True)
print('After we drop non degree seeking students')

# Now df contains only the rows where entrydegreeseeking is not 'Non-Degree undergrad'
# You can continue your analysis using df without saving it to a new CSV file

# Calculate total support across specified years
# df['totalsupport'] = df.loc[:, 'GrantAmountFY2014':'MeritAmountFY2022'].sum(axis=1)
if all(col in df.columns for col in ['GrantAmountFY2014', 'MeritAmountFY2022']):
   df['totalsupport'] = df.loc[:, 'GrantAmountFY2014':'MeritAmountFY2022'].sum(axis=1)
else:
   print("Required columns for totalsupport calculation are missing.")

# Indicate if supported in FY2014
df['supportedY1'] = df.apply(lambda x: 1 if (x['GrantAmountFY2014'] + x['MeritAmountFY2014'] != 0) else 0, axis=1)

# Indicate if supported at all based on 'totalsupport'
df['supported'] = df.apply(lambda x: 1 if (x['totalsupport'] != 0) else 0, axis=1)

# Indicate if graduated
df['grdnum'] = df.apply(lambda x: 1 if (x['Graduated'] == 'Yes') else 0, axis=1)
print('Number of Students: ', len(df.index));
print('Number of Stem Students: ', (df.Stem1.values == 'Yes').sum());
#print(df.head(5));
#print('StemGradRates')
print ('STEM Graduates: ', len(df[(df['Stem1'] == 'Yes') & (df['Graduated'] == 'Yes')]))


After we drop non degree seeking students
Number of Students:  28960
Number of Stem Students:  12509
STEM Graduates:  7660


In [ ]:
# Group students by EFC category and Year 1 support status
efc_support_total_students = df.groupby(['EFC_Category', 'supportedY1']).size()
efc_support_stem_entrants = df[df['Stem1'] == 'Yes'].groupby(['EFC_Category', 'supportedY1']).size()
efc_support_stem_completed = stem_to_stem_df.groupby(['EFC_Category', 'supportedY1']).size()

# Calculate STEM Entry Rate from Total Students
stem_entry_rate_from_total = (efc_support_stem_entrants / efc_support_total_students) * 100

# Calculate STEM Completion Rate from STEM Entrants
stem_completion_rate_from_entrants = (efc_support_stem_completed / efc_support_stem_entrants) * 100

# Calculate STEM Completion Rate from Total Students
stem_completion_rate_from_total = (efc_support_stem_completed / efc_support_total_students) * 100

# Create a DataFrame for the calculations
efc_stem_analysis_df = pd.DataFrame({
    'Total_Students': efc_support_total_students,
    'STEM_Entrants': efc_support_stem_entrants,
    'STEM_To_Stem_Completed': efc_support_stem_completed,
    'STEM_Entry_Rate_From_Total': stem_entry_rate_from_total,
    #'STEM_To_Stem_Completion_Rate_From_Entrants': stem_completion_rate_from_entrants,
    #'STEM_To_Stem_Completion_Rate_From_Total': stem_completion_rate_from_total
}).fillna(0)  # Fill NaN values with 0

efc_stem_analysis_df


Total_Students  STEM_Entrants  \
EFC_Category supportedY1                                  
High         0                       466            159   
             1                      9353           3603   
Low          0                      2712           1048   
             1                      5065           2717   
Medium       0                       974            347   
             1                      1597            720   
Unknown      0                     10439           5557   

                          STEM_To_Stem_Completed  STEM_Entry_Rate_From_Total  
EFC_Category supportedY1                                                      
High         0                                42                   34.120172  
             1                              1290                   38.522399  
Low          0                               390                   38.643068  
             1                              1423                   53.642646  
Medium       0                               109                   35.626283  
             1                               349                   45.084534  
Unknown      0                              1355                   53.233068

In [ ]:
# Function to categorize EFC values, including handling nulls
def categorize_efc(efc):
    if pd.isnull(efc):
        return 'Unknown'  # Handle null values explicitly
    elif efc < 6657 :
        return 'High'
    elif 6657 <= efc <= 13312 :
        return 'Medium'
    else:  # EFC > 13312
        return 'Low'

# Apply the function to the EFC column to create a new category column
df['EFC_Category'] = df['EFC'].apply(categorize_efc)
print(df.head(5));




    campusid   opeid  FirstTimeStatus  Race_New  AttendStatus  Gender AppZip  \
0  @00147352  210400                5         1             2       2  20705   
1  @00159764  210400                5         3             1       2  20876   
2  @00170784  210400                5         9             2       2  21211   
3  @00186283  210400                5         5             2       2  20852   
4  @00191705  210400                5         5             1       2  20794   

   EntryMajor  EntryDegree  FY2014Fall  ...  AttendStatus.1  Status_Desc  \
0      120300           40      2014.0  ...               2    Part-Time   
1      121300           40      2014.0  ...               1    Full-Time   
2      120300           40      2014.0  ...               2    Part-Time   
3      120300           40      2014.0  ...               2    Part-Time   
4      120300           40      2014.0  ...               1    Full-Time   

   degree  entrydegreeseeking  Graduated  Stem1  supportedY1  

In [ ]:
efc_category_counts = df["EFC_Category"].value_counts()
print(efc_category_counts)

EFC_Category
Unknown    10439
High        9819
Low         7777
Medium      2571
Name: count, dtype: int64


In [ ]:
# Ensure totalsupport is in a numeric format, in case it's not
df['totalsupport'] = pd.to_numeric(df['totalsupport'], errors='coerce')

#check the shape of the data
print(df.shape)

# Drop rows where totalsupport is NaN after conversion
df.dropna(subset=['totalsupport'], inplace=True)
print(len(df.index))

# Define bins for support levels
bins = [0,1, 5000, 10000, 15000, 20000, np.inf]
labels = ['NoSup','<5K', '5K-10K', '10K-15K', '15K-20K', '>20K']
df['support_bin'] = pd.cut(df['totalsupport'], bins=bins, labels=labels, right=False)
#print(df.head(5));
# Count the number of students in each bin
student_counts = df['support_bin'].value_counts().sort_index()

# Print the results
print('Students in different support bins:\n', student_counts)

(28960, 107)
28960
Students in different support bins:
 support_bin
NoSup      12096
<5K         4514
5K-10K      2794
10K-15K     1888
15K-20K     1547
>20K        6121
Name: count, dtype: int64


In [ ]:
# Filter students who started and ended in STEM
stem_to_stem_df = df[
    (df['EntryMajor'].astype(str).str.startswith(tuple(hegisstem))) &
    (df['HegisDeg1'].astype(str).str.startswith(tuple(hegisstem))) &
    (df['Graduated'] == 'Yes')
]
stem_allstem_df = df[
    (df['HegisDeg1'].astype(str).str.startswith(tuple(hegisstem))) &
    (df['Graduated'] == 'Yes')
]
stem_graduates = len(df[(df['Stem1'] == 'Yes') & (df['Graduated'] == 'Yes')])


# Count calculations
total_students = len(df)  # Total students in dataset
total_stem_students = (df['Stem1'] == 'Yes').sum()  # Total students who entered in STEM
stem_to_stem_count = len(stem_to_stem_df)  # Students who started & ended in STEM
#stem_graduates = len(df[(df['Stem1'] == 'Yes') & (df['Graduated'] == 'Yes')])  # STEM students who graduated
stem_all=len(stem_allstem_df )

# Percentage calculations
percentage_of_total_students = (stem_to_stem_count / total_students) * 100
percentage_of_stem_students = (stem_to_stem_count / total_stem_students) * 100
percentage_of_stem_students_grad = (stem_to_stem_count / stem_graduates) * 100
percentage_of_stem_students_grad_all = (stem_all / total_students) * 100


# Display results
print(f"Total Students: {total_students}")
print(f"Total STEM Entry Students: {total_stem_students}")
print(f"Total STEM grad all: {stem_all}")
print(f"Students Who Started and Ended in STEM: {stem_to_stem_count}")
print(f"Percentage of Total Students: {percentage_of_total_students:.2f}%")
print(f"Percentage of STEM Entry Students: {percentage_of_stem_students:.2f}%")
print(f"Percentage of STEM grad_all: {percentage_of_stem_students_grad_all:.2f}%")

#print(f"Percentage of STEM Graduates (7660): {percentage_of_stem_students_grad:.2f}%")


Total Students: 30606
Total STEM Entry Students: 14151
Total STEM grad all: 5520
Students Who Started and Ended in STEM: 4958
Percentage of Total Students: 16.20%
Percentage of STEM Entry Students: 35.04%
Percentage of STEM grad_all: 18.04%


In [ ]:
# Parallel calculations

# STEM Entry Rate from Total Students
efc_stem_entry_rate_from_total = (efc_stem_entrants / efc_total_students) * 100

# STEM Completion Rate from STEM Entrants in the same category
efc_stem_completion_rate_within_entrants = (efc_stem_completed / efc_stem_entrants) * 100

# STEM Completion Rate from Total Students in the same category
efc_stem_completion_rate_within_total = (efc_stem_completed / efc_total_students) * 100

# Create DataFrame for parallel calculations
parallel_calculations_df = pd.DataFrame({
    'Total_Students': efc_total_students,
    'STEM_Entrants': efc_stem_entrants,
        'STEM_To_Stem_Completed': efc_stem_completed,

    'STEM_Entry_Rate_From_Total': efc_stem_entry_rate_from_total,
    'STEM_Completion_Rate_Within_Entrants': efc_stem_completion_rate_within_entrants,
    'STEM_Completion_Rate_Within_Total': efc_stem_completion_rate_within_total
}).fillna(0)  # Fill NaN with 0 for missing categories

# Display the parallel calculations DataFrame
parallel_calculations_df


,Total_Students,STEM_Entrants,STEM_To_Stem_Completed,STEM_Entry_Rate_From_Total,STEM_Completion_Rate_Within_Entrants,STEM_Completion_Rate_Within_Total
EFC_Category,,,,,,
High,9819,3762,1332,38.313474,35.406699,13.565536
Low,7777,3765,1813,48.411984,48.154050,23.312331
Medium,2571,1067,458,41.501361,42.924086,17.814080
Unknown,10439,5557,1355,53.233068,24.383660,12.980171


In [ ]:
# Calculate STEM Completion Rate from STEM Entrants
stem_completion_rate_from_entrants = (efc_support_stem_completed / efc_support_stem_entrants) * 100

# Calculate STEM Completion Rate from Total Students
stem_completion_rate_from_total = (efc_support_stem_completed / efc_support_total_students) * 100

# Calculate STEM Entry Rate from Total Students
stem_entry_rate_from_total = (efc_support_stem_entrants / efc_support_total_students) * 100

# Create a combined DataFrame for parallel calculations
combined_parallel_calculations_df = pd.DataFrame({
    'Total_Students': efc_support_total_students,
    'STEM_Entrants': efc_support_stem_entrants,
    'STEM_To_Stem_Completed': efc_support_stem_completed,
    'STEM_Entry_Rate_From_Total': stem_entry_rate_from_total,
    'STEM_To_Stem_Completion_Rate_From_Entrants': stem_completion_rate_from_entrants,
    'STEM_To_Stem_Completion_Rate_From_Total': stem_completion_rate_from_total
}).fillna(0)  # Fill NaN values with 0

combined_parallel_calculations_df


Total_Students  STEM_Entrants  STEM_To_Stem_Completed  \
EFC_Category supported                                                          
High         0                     280             95                      12   
             1                    9539           3667                    1320   
Low          0                    2122            814                     274   
             1                    5655           2951                    1539   
Medium       0                     668            241                      62   
             1                    1903            826                     396   
Unknown      0                   10439           5557                    1355   

                        STEM_Entry_Rate_From_Total  \
EFC_Category supported                               
High         0                           33.928571   
             1                           38.442185   
Low          0                           38.360038   
             1                           52.183908   
Medium       0                           36.077844   
             1                           43.405150   
Unknown      0                           53.233068   

                        STEM_To_Stem_Completion_Rate_From_Entrants  \
EFC_Category supported                                               
High         0                                           12.631579   
             1                                           35.996728   
Low          0                                           33.660934   
             1                                           52.151813   
Medium       0                                           25.726141   
             1                                           47.941889   
Unknown      0                                           24.383660   

                        STEM_To_Stem_Completion_Rate_From_Total  
EFC_Category supported                                           
High         0                                         4.285714  
             1                                        13.837929  
Low          0                                        12.912347  
             1                                        27.214854  
Medium       0                                         9.281437  
             1                                        20.809249  
Unknown      0                                        12.980171

In [ ]:
# For Supported (1) group
supported_stem_to_stem_count = parallel_calculations_df.loc[(['High', 'Low', 'Medium'], 1), 'STEM-to-STEM Count'].sum()
supported_stem_entry_count = efc_support_stem_entry_count.loc[(['High', 'Low', 'Medium'], 1)].sum()
supported_stem_total_count = parallel_calculations_df.loc[(['High', 'Low', 'Medium'], 1), 'STEM TOTAL Count'].sum()

# For Not Supported (0) group
not_supported_stem_to_stem_count = parallel_calculations_df.loc[(['High', 'Low', 'Medium'], 0), 'STEM-to-STEM Count'].sum()
not_supported_stem_entry_count = efc_support_stem_entry_count.loc[(['High', 'Low', 'Medium'], 0)].sum()
not_supported_stem_total_count = parallel_calculations_df.loc[(['High', 'Low', 'Medium'], 0), 'STEM TOTAL Count'].sum()

# STEM-to-STEM Count / STEM Entry Count for each group
supported_stem_to_stem_ratio = (supported_stem_to_stem_count / supported_stem_entry_count) * 100
not_supported_stem_to_stem_ratio = (not_supported_stem_to_stem_count / not_supported_stem_entry_count) * 100

# STEM-to-STEM Count / STEM TOTAL Count for each group
supported_stem_to_stem_total_ratio = (supported_stem_to_stem_count / supported_stem_total_count) * 100
not_supported_stem_to_stem_total_ratio = (not_supported_stem_to_stem_count / not_supported_stem_total_count) * 100

print("===== STEM-to-STEM Ratios =====")
print(f"STEM-to-STEM % of STEM Entry (Supported): {supported_stem_to_stem_ratio:.2f}%")
print(f"STEM-to-STEM % of STEM Entry (Not Supported): {not_supported_stem_to_stem_ratio:.2f}%")

print("\n===== STEM-to-STEM Ratios (Total STEM Students) =====")
print(f"STEM-to-STEM % of Total STEM (Supported): {supported_stem_to_stem_total_ratio:.2f}%")
print(f"STEM-to-STEM % of Total STEM (Not Supported): {not_supported_stem_to_stem_total_ratio:.2f}%")


===== STEM-to-STEM Ratios =====
STEM-to-STEM % of STEM Entry (Supported): 44.90%
STEM-to-STEM % of STEM Entry (Not Supported): 30.82%

===== STEM-to-STEM Ratios (Total STEM Students) =====
STEM-to-STEM % of Total STEM (Supported): 19.20%
STEM-to-STEM % of Total STEM (Not Supported): 11.41%


In [ ]:
# Group students by EFC category and Year 1 support status
efc_support_total_students = df.groupby(['EFC_Category', 'supportedY1']).size()
efc_support_stem_entrants = df[df['Stem1'] == 'Yes'].groupby(['EFC_Category', 'supportedY1']).size()
efc_support_stem_completed = stem_to_stem_df.groupby(['EFC_Category', 'supportedY1']).size()

# Calculate STEM Entry Rate from Total Students
stem_entry_rate_from_total = (efc_support_stem_entrants / efc_support_total_students) * 100

# Calculate STEM Completion Rate from STEM Entrants
stem_completion_rate_from_entrants = (efc_support_stem_completed / efc_support_stem_entrants) * 100

# Calculate STEM Completion Rate from Total Students
stem_completion_rate_from_total = (efc_support_stem_completed / efc_support_total_students) * 100

# Create a DataFrame for the calculations
efc_stem_analysis_df = pd.DataFrame({
    'Total_Students': efc_support_total_students,
    'STEM_Entrants': efc_support_stem_entrants,
    'STEM_To_Stem_Completed': efc_support_stem_completed,
    'STEM_Entry_Rate_From_Total': stem_entry_rate_from_total,
    'STEM_To_Stem_Completion_Rate_From_Entrants': stem_completion_rate_from_entrants,
    'STEM_To_Stem_Completion_Rate_From_Total': stem_completion_rate_from_total
}).fillna(0)  # Fill NaN values with 0

efc_stem_analysis_df


,,Total_Students,STEM_Entrants,STEM_To_Stem_Completed,STEM_Entry_Rate_From_Total,STEM_To_Stem_Completion_Rate_From_Entrants,STEM_To_Stem_Completion_Rate_From_Total
EFC_Category,supportedY1,,,,,,
High,1,9771,3714,1320,38.010439,35.541195,13.509364
Low,1,7579,3568,1808,47.077451,50.672646,23.855390
Medium,1,2563,1059,458,41.318767,43.248347,17.869684
Unknown,1,9047,4168,1337,46.070521,32.077735,14.778380
